## Aprendizaje de reglas de asociación

trabajo práctico:

Obtener reglas de asociación entre películas en el dataset movielens, al estilo de: https://rpubs.com/vitidN/203264 pero usando la notebook de reglas de asociación de Valeria
Aplicar diferentes métricas para ordenamiento de las reglas
Hacer un pequeño informe (entre 200 y 500 palabras, puede ser más pero no mucho más) de interpretación de los efectos de las métricas aplicadas

Objetivo:
    
    derivar reglas de la forma {A} -> {B}
    

In [1]:
import numpy as np
from itertools import combinations, groupby
from collections import Counter

In [2]:
import pandas as pd
import numpy as np
import sys
import os
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display

### Recomendación de películas con reglas de asociación

In [3]:
!ls 

ml-20m	   rules_confidence.dat  rules_support.dat
README.md  rules_lift.dat	 tp_reglas_asoc.ipynb


### Películas

In [4]:
#PELICULAS

dir_= 'ml-20m'
f_pelis = os.path.join(dir_, 'movies.csv')
pelis = pd.read_csv(f_pelis)
display(pelis.head())
display(pelis.tail())

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,movieId,title,genres
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)
27277,131262,Innocence (2014),Adventure|Fantasy|Horror


Separar título de año y separar los géneros.

In [5]:
pelis_ = pelis
pelis_['año'] = pelis.title.apply(lambda x: x[-5:-1])
pelis_['titulo'] = pelis.title.apply(lambda x: x[:-7])

In [6]:
pelis_.head()

,movieId,title,genres,año,titulo
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,1995,Father of the Bride Part II


### Ratings

In [7]:
### hay que descargar la base de datos al directorio ml-20m porque es demasiado pesada para subirla al github
f_ratings = os.path.join(dir_, 'ratings.csv')
ratings = pd.read_csv(f_ratings)
print(ratings.shape)                   
ratings.head()

(20000263, 4)


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [8]:
minirating_ = ratings.loc[:2000000] 
print(minirating_.shape)
minirating_.columns

(2000001, 4)


Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

### Definir transacción y items
- Transacción = usuario
- items = películas ranqueadas por cada usuario

In [13]:
#Agregar el nombre de la película en rating

minirating_ = pd.merge(minirating_[['userId','movieId']], pelis_[['movieId','titulo']] ,on='movieId', how= "inner")
display(minirating_.head())
minirating_ = minirating_.sort_values( by=['userId','titulo'], axis=0, ascending=True, inplace=False, 
                                          kind='quicksort', na_position='last')
minirating_upl = minirating_.values[:,[0,2]]

minirating_upl

,userId,movieId,titulo
0,1,924,2001: A Space Odyssey
1,2,924,2001: A Space Odyssey
2,3,924,2001: A Space Odyssey
3,7,924,2001: A Space Odyssey
4,14,924,2001: A Space Odyssey


array([[1, '2001: A Space Odyssey'],
       [1, '28 Days Later'],
       [1, '7th Voyage of Sinbad, The'],
       ...,
       [13567, 'Sphere'],
       [13567, 'Taken'],
       [13567, 'Whole Nine Yards, The']], dtype=object)

In [14]:
transactions=[]
for userId, pelicula in groupby(minirating_upl, lambda x: x[0]):
    transactions.append([item[1] for item in pelicula])

In [15]:
print(len(transactions))

13567


In [16]:
from efficient_apriori import apriori
#wARNING-CUIDADO!! no poner min_support pequeño como por ejempo 0.001!!
#min_support y min_confiden entre 0 y 1  
itemsets, rules = apriori(transactions, min_support=0.01,  min_confidence=0.5, max_length=2)
#minsupport de 0.01 =  tiene que estar en por lo menos 10% de las transacciones. 

In [17]:
#rules = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
rules=sorted(rules, key=lambda rule: rule.confidence, reverse=True)
output = open("rules_confidence.dat","w")
for rule in rules:
    
    pair = (rule.lhs, rule.rhs) 
    items = [x for x in pair]
    output.write("Rule: " + str(items[0]) + " -> " + str(items[1])+"\n")
    output.write("Support: " + str(rule.support)+"\n")
    output.write("Confidence: " + str(rule.confidence)+"\n")
    output.write("Lift: " + str(rule.lift)+"\n")
    output.write("====================================="+"\n")
output.close()    

In [18]:
!head -n 10 rules.dat

head: no se puede abrir 'rules.dat' para lectura: No existe el archivo o el directorio


In [19]:
#rules = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
rules=sorted(rules, key=lambda rule: rule.lift, reverse=True)
output = open("rules_lift.dat","w")
for rule in rules:
    
    pair = (rule.lhs, rule.rhs) 
    items = [x for x in pair]
    output.write("Rule: " + str(items[0]) + " -> " + str(items[1])+"\n")
    output.write("Support: " + str(rule.support)+"\n")
    output.write("Confidence: " + str(rule.confidence)+"\n")
    output.write("Lift: " + str(rule.lift)+"\n")
    output.write("====================================="+"\n")
output.close()    

In [20]:
#rules = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
rules=sorted(rules, key=lambda rule: rule.support, reverse=True)
output = open("rules_support.dat","w")
for rule in rules:
    
    pair = (rule.lhs, rule.rhs) 
    items = [x for x in pair]
    output.write("Rule: " + str(items[0]) + " -> " + str(items[1])+"\n")
    output.write("Support: " + str(rule.support)+"\n")
    output.write("Confidence: " + str(rule.confidence)+"\n")
    output.write("Lift: " + str(rule.lift)+"\n")
    output.write("====================================="+"\n")
output.close()    

### Conclusiones

El tp es un sistema de recomendación que usa reglas de asociación, similar al que se implementa en: https://rpubs.com/vitidN/203264 , usando la base de datos de películas de: MovieLens, 20M (ml-20m). Usé solamente un 10% de las anotaciones (las primeras 2000000 líneas de la base de datos) porque sino demoraba mucho tiempo en terminar el algoritmo. 
Usé el csv de los ranking de usuarios. Cada línea es una anotación con el ranking de una película y el id del usuario que le da la puntuación. La idea es generar reglas entre películas, considerando el conjunto de películas que puntúa cada usuario. Entonces, una transacción es cada usuario (conjunto de películas que califica un usuario), y un item es cada película.
Ordené las reglas de recomendación usando: support, confianza, y lift...

- Si se ordena por soporte, aparecen primero pares de películas taquilleras: es muy probable que los usuarios hayan visto ambas porque son películas populares. Dan soportes del orden de 30%. El lift es relativamente bajo (entre 1 y 2): aparecen juntas porque ambas son muy probables y no porque reflejen una afinidad particular entre las dos. No aportamos mucho info nueva. 
- Si se ordena por confianza (probabilidad condicional de que aparezca una peli B si apareció una peli A), entonces aparecen primero pares donde al menos una de las películas es taquillera y la otra es menos popular (yo por lo menos conozco el título de una y la otra no :) ). Además los soportes de las que tienen mayor confianza es menor al 2%.  Es muy probable que los usuarios hayan visto la película taquillera, dado que vieron la otra, pero la razón es que muchos ven las taquilleras, no porque estén relacionadas especiamente. Entonces no discrimina películas muy relacionadas.
- Si se ordena por lift entonces las primeras que aparecen son precuelas (o secuelas), y se ven algunas reglas entre películas de superhéroes. También aparecen remakes como jean de florette y manon de source.  